In [1]:
import math
import random
import feedback as fb

In [2]:

class AdPublisher( fb.Component ):

    def __init__( self, scale, min_price, relative_width=0.1 ):
        self.scale = scale
        self.min = min_price
        self.width = relative_width

    def work( self, u ):
        if u <= self.min:       # Price below min: no impressions
            return 0

        # "demand" is the number of impressions served per day
        # The demand is modeled (!) as Gaussian distribution with
        # a mean that depends logarithmically on the price u.
        
        mean = self.scale*math.log( u/self.min )        
        demand = int( random.gauss( mean, self.width*mean ) )

        return max( 0, demand ) # Impression demand is greater than zero


class AdPublisherWithWeekend( AdPublisher ):
    
    def __init__( self, weekday, weekend, min_price, relative_width=0.1 ):
        AdPublisher.__init__( self, None, min_price, relative_width )

        self.weekday = weekday
        self.weekend = weekend        

        self.t = 0      # Internal day counter

    def work( self, u ):
        self.t += 1

        if self.t%7 < 2: # Weekend
            self.scale = self.weekend
        else:
            self.scale = self.weekday

        return AdPublisher.work( self, u )
    


In [4]:

def statictest():
    fb.static_test( AdPublisher, (100,2), 20, 100, 10, 5000 )


def closedloop( kp, ki, f=fb.Identity() ):
    def setpoint( t ):
        if t > 1000:
            return 125
        return 100

    k = 1.0/20.0

    p = AdPublisher( 100, 2 )
    c = fb.PidController( k*kp, k*ki )

    fb.closed_loop( setpoint, c, p, returnfilter=f )


accumul_goal = 0
def closedloop_accumul( kp, ki ):
    def setpoint( t ):
        global accumul_goal
        
        if t > 1000:
            accumul_goal += 125
        else:
            accumul_goal += 100
        return accumul_goal
    
    k = 1.0/20.0

    p = AdPublisher( 100, 2 )
    c = fb.PidController( k*kp, k*ki )

    fb.closed_loop( setpoint, c, p, returnfilter=fb.Integrator() )


def specialsteptest():
    p = AdPublisher( 100, 2 )
    f = fb.RecursiveFilter(0.05)
    
    for t in range( 500 ):
        r = 5.50
        u = r
        y = p.work( u )
        z = f.work( y )

        print(t, t*fb.DT, r, 0, u, u, y, z, p.monitoring())
        
    quit()
    



In [9]:
fb.DT = 1

#statictest()

closedloop( 0.5, 0.25 ) # default

#closedloop( 0.0, 0.25 ) # w/o prop ctrl
#closedloop( 0.0, 1.75 )  # ringing   
#closedloop( 1.0, 0.125, fb.RecursiveFilter(0.125) )
        
#closedloop_accumul( 0.5, 0.125 )




0 0 100 100 3.75 3.75 66 66 
1 1 100 34 2.5250000000000004 2.5250000000000004 20 20 
2 2 100 80 4.675000000000001 4.675000000000001 66 66 
3 3 100 34 3.95 3.95 70 70 
4 4 100 30 4.225 4.225 70 70 
5 5 100 30 4.6 4.6 73 73 
6 6 100 27 4.8625 4.8625 91 91 
7 7 100 9 4.5249999999999995 4.5249999999999995 86 86 
8 8 100 14 4.825 4.825 81 81 
9 9 100 19 5.1875 5.1875 92 92 
10 10 100 8 5.0125 5.0125 92 92 
11 11 100 8 5.112500000000001 5.112500000000001 85 85 
12 12 100 15 5.4750000000000005 5.4750000000000005 97 97 
13 13 100 3 5.2125 5.2125 106 106 
14 14 100 -6 4.9125 4.9125 100 100 
15 15 100 0 5.0625 5.0625 93 93 
16 16 100 7 5.325 5.325 100 100 
17 17 100 0 5.15 5.15 96 96 
18 18 100 4 5.3 5.3 99 99 
19 19 100 1 5.237500000000001 5.237500000000001 79 79 
20 20 100 21 6.000000000000001 6.000000000000001 115 115 
21 21 100 -15 4.9125000000000005 4.9125000000000005 82 82 
22 22 100 18 5.9625 5.9625 121 121 
23 23 100 -21 4.725 4.725 86 86 
24 24 100 14 5.775 5.775 105 105 
25 25 100 -5 5

NameError: name 'quit' is not defined